In [40]:
import os
import sys
import numpy as np
import csv
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [75]:
df_train = pd.read_pickle("./train_mr.pkl")
df_test = pd.read_pickle("./test_mr.pkl")
pd.set_option('display.max_columns', 500)

In [73]:
df_train.head()

,acquirer,bank,card,money,trade_cat,coin,online,trade_type,fallback,3ds,fraud_ind,pay_type,install,term,date,time,mcc,shop,excess,city,nation,status,key
0,6716,12765,101514,966.580017,4,62,0,2,0.0,0.0,0,5,1,2,1,29.0,247,34949,0,5817,102,0,1119159
1,5575,34837,60869,1071.099976,5,62,1,8,0.0,0.0,0,5,0,0,1,40.0,203,33794,0,2138,44,0,641444
2,6769,22630,110573,438.209991,5,62,0,5,0.0,0.0,0,5,0,0,1,45.0,251,79257,0,3588,102,0,188670
3,6767,91588,208961,1174.170044,5,62,0,5,0.0,0.0,0,5,0,0,1,48.0,293,14765,0,5817,102,0,1475737
4,3198,84148,180103,367.290009,5,60,0,5,0.0,0.0,0,5,0,0,1,51.0,251,68966,0,3982,38,0,641443


# Feature Engineering

## 1

In [107]:
#money / term groupby high_category: max, min, mean, std, var, median, median_diff, quantiles
def money_term_inf(dataframe,c,b):
    #b grouped by c
    a = dataframe.copy()
    a[b + '_' + c + '_max'] = a[c].map(a.loc[:, [b, c]].groupby(c).max().loc[:, b].to_dict())
    a[b + '_' + c + '_min'] = a[c].map(a.loc[:, [b, c]].groupby(c).min().loc[:, b].to_dict())
    a[b + '_' + c + '_mean'] = a[c].map(a.loc[:, [b, c]].groupby(c).mean().loc[:, b].to_dict())
    a[b + '_' + c + '_std'] = a[c].map(a.loc[:, [b, c]].groupby(c).std().loc[:, b].to_dict())
    a[b + '_' + c + '_var'] = a[b + '_' + c + '_std']**2
    a[b + '_' + c + '_median'] = a[c].map(a.loc[:, [b, c]].groupby(c).median().loc[:, b].to_dict())
    a[b + '_' + c + '_median_diff'] = a[b] - a[b + '_' + c + '_median']
    return a

## 2

In [114]:
#card_num: one bank has how many cards. ( 一個帳號有幾張卡 )
def card_num(dataframe):
    a = dataframe.copy()
    a['card_num'] = a['bank'].map(a.loc[:, ['card', 'bank']].groupby('bank').count().loc[:,'card'].to_dict())
    return a

## 4

In [167]:
#dominant coin type 貨幣在那個國家交易過幾次
def coin_trade_country_per(dataframe):
    a = dataframe.copy()
    dct =(tmp.groupby(['nation','coin']).size()/tmp.groupby('nation').size()).to_dict()
    a['coin_country_per'] = a.set_index(['nation', 'coin']).index.map(dct.get)
    return a

# 跑Function

In [109]:
#跑money_term_inf
high_category = ['acquirer', 'bank', 'card', 'coin', 'mcc', 'shop', 'city', 'nation']
for i in (high_category):
    df_train  =money_term_inf(df_train,i,'money')
    df_train = money_term_inf(df_train,i,'term')

In [115]:
#跑card_num
df_train = card_num(df_train)

In [168]:
#跑coin_trade_country_per
df_train = coin_trade_country_per(df_train)

In [170]:
df_train.head(500)

,acquirer,bank,card,money,trade_cat,coin,online,trade_type,fallback,3ds,fraud_ind,pay_type,install,term,date,time,mcc,shop,excess,city,nation,status,key,money_acquirer_max,money_acquirer_min,money_acquirer_mean,money_acquirer_std,money_acquirer_var,money_acquirer_median,money_acquirer_median_diff,term_acquirer_max,term_acquirer_min,term_acquirer_mean,term_acquirer_std,term_acquirer_var,term_acquirer_median,term_acquirer_median_diff,money_bank_max,money_bank_min,money_bank_mean,money_bank_std,money_bank_var,money_bank_median,money_bank_median_diff,term_bank_max,term_bank_min,term_bank_mean,term_bank_std,term_bank_var,term_bank_median,term_bank_median_diff,money_card_max,money_card_min,money_card_mean,money_card_std,money_card_var,money_card_median,money_card_median_diff,term_card_max,term_card_min,term_card_mean,term_card_std,term_card_var,term_card_median,term_card_median_diff,money_coin_max,money_coin_min,money_coin_mean,money_coin_std,money_coin_var,money_coin_median,money_coin_median_diff,term_coin_max,term_coin_min,term_coin_mean,term_coin_std,term_coin_var,term_coin_median,term_coin_median_diff,money_mcc_max,money_mcc_min,money_mcc_mean,money_mcc_std,money_mcc_var,money_mcc_median,money_mcc_median_diff,term_mcc_max,term_mcc_min,term_mcc_mean,term_mcc_std,term_mcc_var,term_mcc_median,term_mcc_median_diff,money_shop_max,money_shop_min,money_shop_mean,money_shop_std,money_shop_var,money_shop_median,money_shop_median_diff,term_shop_max,term_shop_min,term_shop_mean,term_shop_std,term_shop_var,term_shop_median,term_shop_median_diff,money_city_max,money_city_min,money_city_mean,money_city_std,money_city_var,money_city_median,money_city_median_diff,term_city_max,term_city_min,term_city_mean,term_city_std,term_city_var,term_city_median,term_city_median_diff,money_nation_max,money_nation_min,money_nation_mean,money_nation_std,money_nation_var,money_nation_median,money_nation_median_diff,term_nation_max,term_nation_min,term_nation_mean,term_nation_std,term_nation_var,term_nation_median,term_nation_median_diff,card_num,coin_country_per
0,6716,12765,101514,966.580017,4,62,0,2,0.0,0.0,0,5,1,2,1,29.0,247,34949,0,5817,102,0,1119159,4307.660156,0.000000,805.271362,449.041382,201638.162601,722.380005,244.200012,8,0,0.248020,0.794906,0.631876,0,2,1248.109985,123.239998,800.699097,384.506927,147845.577288,966.580017,0.000000,2,0,1.090909,0.943880,0.890909,1.0,1.0,1248.109985,123.239998,784.799011,401.475952,1.611829e+05,966.580017,0.000000,2,0,1.100000,0.994429,0.988889,1.5,0.5,7208.770020,0.0,682.752686,410.813324,168767.587155,644.659973,321.920044,8,0,0.060374,0.401904,0.161527,0,2,4910.180176,0.000000,765.138977,376.367615,141652.581430,708.229980,258.350037,8,0,0.188447,0.682097,0.465256,0,2,3140.280029,438.209991,923.810547,265.421509,70448.577328,865.039978,101.540039,7,0,1.476157,0.911971,0.831691,1.0,1.0,7208.770020,0.000000,680.749023,390.326477,152354.758687,636.820007,329.760010,8,0,0.029591,0.261273,0.068264,0.0,2.0,7208.770020,0.0,673.943787,387.901794,150467.802125,607.219971,359.360046,8,0,0.057499,0.392571,0.154112,0,2,11,0.858183
1,5575,34837,60869,1071.099976,5,62,1,8,0.0,0.0,0,5,0,0,1,40.0,203,33794,0,2138,44,0,641444,3089.750000,0.000000,641.221802,593.991699,352826.138741,618.184998,452.914978,0,0,0.000000,0.000000,0.000000,0,0,1535.599976,168.199997,615.295532,304.744781,92869.381848,513.799988,557.299988,0,0,0.000000,0.000000,0.000000,0.0,0.0,1535.599976,1071.099976,1303.349976,328.451111,1.078801e+05,1303.349976,-232.250000,0,0,0.000000,0.000000,0.000000,0.0,0.0,7208.770020,0.0,682.752686,410.813324,168767.587155,644.659973,426.440002,8,0,0.060374,0.401904,0.161527,0,0,4035.129883,0.000000,1178.297852,743.321838,552527.355411,1216.844971,-145.744995,4,0,0.134211,0.534582,0.285778,0,0,2951.280029,0.000000,613.450684,617.167969,380896.301651,565.219971,505.880005,0,0,0.000000,0.000000,0.000000,0.0,0.0,2951.280029,0.000000,613.549072,616.545715,380128.619094,566.229980,504.869995,0,0,0.000000,0.000000,0.000000,0.0,0.0,2

In [165]:
tmp.head()

,acquirer,bank,card,money,trade_cat,coin,online,trade_type,fallback,3ds,fraud_ind,pay_type,install,term,date,time,mcc,shop,excess,city,nation,status,key,money_acquirer_max,money_acquirer_min,money_acquirer_mean,money_acquirer_std,money_acquirer_var,money_acquirer_median,money_acquirer_median_diff,term_acquirer_max,term_acquirer_min,term_acquirer_mean,term_acquirer_std,term_acquirer_var,term_acquirer_median,term_acquirer_median_diff,money_bank_max,money_bank_min,money_bank_mean,money_bank_std,money_bank_var,money_bank_median,money_bank_median_diff,term_bank_max,term_bank_min,term_bank_mean,term_bank_std,term_bank_var,term_bank_median,term_bank_median_diff,money_card_max,money_card_min,money_card_mean,money_card_std,money_card_var,money_card_median,money_card_median_diff,term_card_max,term_card_min,term_card_mean,term_card_std,term_card_var,term_card_median,term_card_median_diff,money_coin_max,money_coin_min,money_coin_mean,money_coin_std,money_coin_var,money_coin_median,money_coin_median_diff,term_coin_max,term_coin_min,term_coin_mean,term_coin_std,term_coin_var,term_coin_median,term_coin_median_diff,money_mcc_max,money_mcc_min,money_mcc_mean,money_mcc_std,money_mcc_var,money_mcc_median,money_mcc_median_diff,term_mcc_max,term_mcc_min,term_mcc_mean,term_mcc_std,term_mcc_var,term_mcc_median,term_mcc_median_diff,money_shop_max,money_shop_min,money_shop_mean,money_shop_std,money_shop_var,money_shop_median,money_shop_median_diff,term_shop_max,term_shop_min,term_shop_mean,term_shop_std,term_shop_var,term_shop_median,term_shop_median_diff,money_city_max,money_city_min,money_city_mean,money_city_std,money_city_var,money_city_median,money_city_median_diff,term_city_max,term_city_min,term_city_mean,term_city_std,term_city_var,term_city_median,term_city_median_diff,money_nation_max,money_nation_min,money_nation_mean,money_nation_std,money_nation_var,money_nation_median,money_nation_median_diff,term_nation_max,term_nation_min,term_nation_mean,term_nation_std,term_nation_var,term_nation_median,term_nation_median_diff,card_num,coin_country_num
0,6716,12765,101514,966.580017,4,62,0,2,0.0,0.0,0,5,1,2,1,29.0,247,34949,0,5817,102,0,1119159,4307.660156,0.0,805.271362,449.041382,201638.162601,722.380005,244.200012,8,0,0.248020,0.794906,0.631876,0,2,1248.109985,123.239998,800.699097,384.506927,147845.577288,966.580017,0.000000,2,0,1.090909,0.94388,0.890909,1.0,1.0,1248.109985,123.239998,784.799011,401.475952,161182.940153,966.580017,0.000000,2,0,1.1,0.994429,0.988889,1.5,0.5,7208.770020,0.0,682.752686,410.813324,168767.587155,644.659973,321.920044,8,0,0.060374,0.401904,0.161527,0,2,4910.180176,0.000000,765.138977,376.367615,141652.581430,708.229980,258.350037,8,0,0.188447,0.682097,0.465256,0,2,3140.280029,438.209991,923.810547,265.421509,70448.577328,865.039978,101.540039,7,0,1.476157,0.911971,0.831691,1.0,1.0,7208.770020,0.000000,680.749023,390.326477,152354.758687,636.820007,329.760010,8,0,0.029591,0.261273,0.068264,0.0,2.0,7208.770020,0.0,673.943787,387.901794,150467.802125,607.219971,359.360046,8,0,0.057499,0.392571,0.154112,0,2,11,0.858183
1,5575,34837,60869,1071.099976,5,62,1,8,0.0,0.0,0,5,0,0,1,40.0,203,33794,0,2138,44,0,641444,3089.750000,0.0,641.221802,593.991699,352826.138741,618.184998,452.914978,0,0,0.000000,0.000000,0.000000,0,0,1535.599976,168.199997,615.295532,304.744781,92869.381848,513.799988,557.299988,0,0,0.000000,0.00000,0.000000,0.0,0.0,1535.599976,1071.099976,1303.349976,328.451111,107880.132212,1303.349976,-232.250000,0,0,0.0,0.000000,0.000000,0.0,0.0,7208.770020,0.0,682.752686,410.813324,168767.587155,644.659973,426.440002,8,0,0.060374,0.401904,0.161527,0,0,4035.129883,0.000000,1178.297852,743.321838,552527.355411,1216.844971,-145.744995,4,0,0.134211,0.534582,0.285778,0,0,2951.280029,0.000000,613.450684,617.167969,380896.301651,565.219971,505.880005,0,0,0.000000,0.000000,0.000000,0.0,0.0,2951.280029,0.000000,613.549072,616.545715,380128.619094,566.229980,504.869995,0,0,0.000000,0.000000,0.000000,0.0,0.0,2951.280029,0.0,613.7